In [2]:
import pandas as pd
import numpy as np
import torch
import sklearn

### Importing data

In [7]:
from sklearn.model_selection import train_test_split
train = pd.read_csv('data/train.csv')
df = train.dropna(axis=0, how='any') # figure out how to bootstrap
x = df.drop(['Credit_Score'], axis=1).values
y = df['Credit_Score'].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, train_size=0.7)

/var/folders/k0/8ymf_pwj26lfhg2zckp9zy_w0000gp/T/ipykernel_36858/982677478.py:2: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('data/train.csv')


### Decision Tree

In [10]:
from sklearn import tree, metrics
import seaborn as sns 
import matplotlib.pyplot as plt

# makes a decision tree using x/y train data
dtc = tree.DecisionTreeClassifier().fit(x_train, y_train)

# use model for pred
y_pred_dtc = dtc.predict(x_test)

# getting statistics
accuracy = metrics.accuracy_score(y_test, y_pred_dtc)
precision = metrics.precision_score(y_test, y_pred_dtc)
recall = metrics.recall_score(y_test, y_pred_dtc)
f1 = metrics.f1_score(y_test, y_pred_dtc)
cm = metrics.confusion_matrix(y_test, y_pred_dtc)

print(f'accuracy : {accuracy}')
print(f'precision : {precision}')
print(f'recall : {recall}')
print(f'f1 : {f1}')
print(cm)






ValueError: could not convert string to float: '0x5210'